In [9]:
import cv2

In [10]:
haar=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

In [11]:
import os
#os.mkdir("faces\\train")

In [14]:
a=input("enter name : ")
try:
    import os
    os.mkdir('C:\\Users\\shubham\\faces\\train\\'+a)
    os.mkdir('C:\\Users\\shubham\\faces\\test\\'+a)
    
    video=cv2.VideoCapture(0)
    j=1
    count=1
    while count<51:
        ret,frame1=video.read()
        faces =haar.detectMultiScale(frame1, scaleFactor=1.1,minNeighbors=5)
        human=1
        for(x,y,w,h) in faces:
            cv2.rectangle(frame1,(x,y),(x+w,y+h),(250,0,0),10)
            cv2.putText(frame1,"p"+str(human),(x-10,y-10),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,255,0),5)
            human+=1
        cv2.imshow("video capture",frame1)
        cv2.waitKey(1)
        l=len(faces)
        for i in range(l):
                y,x,w,h=faces[i][0],faces[i][1],faces[i][2],faces[i][3]
                crop=frame1[x:x+h,y:y+w]
                if count<15:
                    cv2.imshow("face",crop)
                    cv2.waitKey(1)
                    cv2.imwrite('C:\\Users\\shubham\\faces\\test\\'+a+'/'+str(a)+str(j)+".jpg",crop)
                    count+=1
                else :
                    cv2.imshow("face",crop)
                    cv2.waitKey(1)
                    cv2.imwrite('C:\\Users\\shubham\\faces\\train\\'+a+'/'+str(a)+str(j)+".jpg",crop)
                    count+=1
                    
                #cv2.putText(crop,"p"+str(human),(x-10,y-10),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,255,0),5)
            #human+=1
                j=j+1
         
    video.release()
    cv2.destroyAllWindows()
    
except :
    print("already exist")        

enter name : shubham
already exist


In [11]:
video.release()

In [12]:
label=[]
for file in os.listdir("faces\\train"):
    label.append(file)

In [13]:
d={}
for i in range(len(label)):
    d[i]=label[i]
d

{0: 'akanksha jangid',
 1: 'Osheen',
 2: 'piyush',
 3: 'ronish',
 4: 'sahil',
 5: 'sanket',
 6: 'sarvagya',
 7: 'satyam',
 8: 'utkarsh'}

In [14]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
import keras

Using TensorFlow backend.


In [15]:
classifier=Sequential()

In [16]:
classifier.add(Conv2D(32,(5,5),input_shape=(64,64,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Conv2D(64,(5,5),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())

Instructions for updating:
Colocations handled automatically by placer.


In [17]:
classifier.add(Dense(units=800,activation='relu'))
classifier.add(Dense(units=400,activation='relu'))
classifier.add(Dense(units=len(label),activation='softmax'))

In [18]:
print(len(label))

9


In [19]:
classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [20]:
from keras.preprocessing.image import ImageDataGenerator

In [21]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [22]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [23]:
train_set=train_datagen.flow_from_directory(
    'faces\\train',
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical')
#len(train_set)


Found 324 images belonging to 9 classes.


In [24]:
test_set=train_datagen.flow_from_directory(
    'faces\\test',
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical')

Found 126 images belonging to 9 classes.


In [25]:
classifier.fit_generator(
        train_set,
        steps_per_epoch=216,
        epochs=2,
        validation_data=test_set,
        validation_steps=84)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
216/216 [==============================] - 193s 895ms/step - loss: 0.2111 - acc: 0.9239 - val_loss: 0.6689 - val_acc: 0.8923
Epoch 2/2
216/216 [==============================] - 172s 797ms/step - loss: 8.0501e-05 - acc: 1.0000 - val_loss: 0.6497 - val_acc: 0.8991


In [26]:
classifier.save('model4_categorical_complex.h5')

In [27]:
from keras.models import load_model
a=load_model('model4_categorical_complex.h5')

In [28]:
import numpy as n
from keras.preprocessing import image


In [29]:
test_image = image.load_img('faces\\test\\Osheen\\Osheen1.jpg',target_size = (64, 64)) #
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
#plt.figure(1)
plt.imshow(test_image)
plt.show()

<Figure size 640x480 with 1 Axes>

In [30]:
train_set.class_indices

{'Osheen': 0,
 'akanksha jangid': 1,
 'piyush': 2,
 'ronish': 3,
 'sahil': 4,
 'sanket': 5,
 'sarvagya': 6,
 'satyam': 7,
 'utkarsh': 8}

In [31]:
test_image = image.img_to_array(test_image)
test_image = n.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)

In [32]:
a=result.argmax()

In [33]:
s=train_set.class_indices
name=[]
for i in s:
    name.append(i)
for i in range(len(s)):
    if i==a:
        q=name[i]
        print(q)

sahil


In [34]:
def predictor(path):
    test_image = image.load_img(path,target_size = (64, 64)) #
    test_image = image.img_to_array(test_image)
    test_image = n.expand_dims(test_image, axis = 0)
    result = classifier.predict(test_image)
    a=result.argmax()
    s=train_set.class_indices
    name=[]
    for i in s:
        name.append(i)
    for i in range(len(s)):
        if i==a:
            q=name[i]
    return(q)
    

In [ ]:
video=cv2.VideoCapture(0)
y=None
while True:
    ret,frame1=video.read()
    faces =haar.detectMultiScale(frame1, scaleFactor=1.1,minNeighbors=5)
    
    if cv2.waitKey(20) & 0xff == 32:
        break
    for (x,y,w,h) in faces:
        w_rm=int(0.2*w/2)
        cv2.rectangle(frame1,(x,y),(x+w,y+h),(250,0,0),10)
        path='faces\\a.jpg'
        crop=frame1[y:y+h,x+w_rm:x+w-w_rm]
        cv2.imwrite(path,crop)
        
        name1= predictor(path)
        cv2.putText(frame1,name1,(x-10,y-10),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,255,0),5)
    cv2.imshow('detector',frame1)
cv2.destroyAllWindows()
video.release()

In [ ]:
video.release()

In [ ]:
a=input("enter name")
haar=cv2.CascadeClassifier("G://programs//New Folder//haarcascade_frontalface_alt.xml")
import os
os.mkdir('faces\\train\\'+a)
os.mkdir('faces\\test\\'+a)

video=cv2.VideoCapture(0)
j=1
count=1
while count<51:
    ret,frame1=video.read()
    faces =haar.detectMultiScale(frame1, scaleFactor=1.1,minNeighbors=5)
    human=1
    for(x,y,w,h) in faces:
        cv2.rectangle(frame1,(x,y),(x+w,y+h),(250,0,0),10)
        cv2.putText(frame1,"p"+str(human),(x-10,y-10),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,255,0),5)
        human+=1
    cv2.imshow("video capture",frame1)
    cv2.waitKey(1)
    l=len(faces)
    for i in range(l):
            y,x,w,h=faces[i][0],faces[i][1],faces[i][2],faces[i][3]
            crop=frame1[x:x+h,y:y+w]
            if count<15:
                cv2.imshow("face",crop)
                cv2.waitKey(1)
                cv2.imwrite('faces\\test\\'+a+'/'+str(a)+str(j)+".jpg",crop)
                count+=1
            else :
                cv2.imshow("face",crop)
                cv2.waitKey(1)
                cv2.imwrite('faces\\train\\'+a+'/'+str(a)+str(j)+".jpg",crop)
                count+=1

            #cv2.putText(crop,"p"+str(human),(x-10,y-10),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,255,0),5)
        #human+=1
            j=j+1

video.release()
cv2.destroyAllWindows()
